In [23]:
#!conda install -c conda-forge geopy --yes
#!pip install lxml html5lib beautifulsoup4
#!conda install -c conda-forge geocoder -y
#!conda install -c conda-forge folium=0.5.0 --yes

We first start off with installing the required libraries

In [61]:
import pandas as pd
import numpy as np
import folium
from geopy.geocoders import Nominatim
import requests
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

Importing the required libraries for the code to execute

In [4]:
url= 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs=pd.read_html(url)

df1=dfs[0]

df= df1[['Postal Code', 'Borough', 'Neighbourhood']]

df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


The code above scraps the Wikipedia URL and extracts Toronto's postal codes, boroughs and neighbourhoods. This will be used for our analysis

In [5]:
df1=df[df.Borough != 'Not assigned']

df1.shape

(103, 3)

The lines above ensure that the boroughs with 'Not assigned' are dropped from our dataframe

In [15]:
!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [16]:
df3 = pd.read_csv('Geospatial_Coordinates.csv')
df3

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Unfortunatelly the geocode did not work for me so I resolve the issue by downloading the csv directly

In [19]:
df5=pd.merge(df1,df3)

df5

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


.merge is a powerful tool that automatically join two data frames and sort them based on a common column. In this case it was the postal code columns.

In [59]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


map_tr = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df5['Latitude'], df5['Longitude'], df5['Borough'], df5['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tr)  
    
map_tr

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


The map shows the boroughs of toronto and how they are clustered together. Using the coordinates and the imported library of folium 

In [84]:
tr_data = df5[df5['Borough'] == 'York'].reset_index(drop=True)
tr_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M6C,York,Humewood-Cedarvale,43.693781,-79.428191
1,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512
2,M6M,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.691116,-79.476013
3,M6N,York,"Runnymede, The Junction North",43.673185,-79.487262
4,M9N,York,Weston,43.706876,-79.518188


In [106]:
CLIENT_ID = 'OJ4HUXL1DFTO1JBWCJHVKOATZ4N1IKVB5ICSMCQPX4SOUUGH' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

In [88]:
tr_data.loc[0, 'Neighbourhood']

neighbourhood_latitude = tr_data.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = tr_data.loc[0, 'Longitude'] # neighborhood longitude value

tr_name = tr_data.loc[0, 'Neighbourhood'] # neighborhood name

radius=5000

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))


Latitude and longitude values of Islington Avenue, Humber Valley Village are 43.6937813, -79.42819140000002.


In [99]:
url1 = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, 500, 100)

results = requests.get(url1).json()
#results

In [100]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [105]:
nearby_venues=results['response']['venue']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

KeyError: 'venue'

I tried to get the coding running for exploring the Toronto's neighbourhoods, but unfortunatelly i couldnt get the code to work.

I treid different solutions from the discussion form.

If it did work, I would look at different venues and try clustering them together in the same manner as the New York lab